## https://github.com/Nixtla/hierarchicalforecast/blob/main/nbs/examples/TourismSmall.ipynb

In [ ]:
# %#pip install statsforecast hierarchicalforecast utilsforcast

In [4]:
from statsforecast.core import StatsForecast
from statsforecast.models import AutoARIMA, Naive, HoltWinters
import pandas as pd

from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.evaluation import HierarchicalEvaluation, mse, rel_mse, mqloss
from hierarchicalforecast.methods import TopDown, BottomUp, MinTrace
from hierarchicalforecast.utils import aggregate, HierarchicalPlot
from utilsforecast.preprocessing import fill_gaps
import numpy as np

import pickle
from datetime import date, datetime
from sklearn.metrics import mean_absolute_percentage_error
from qgridnext import show_grid
from importlib import reload
import re
import pickle

C:\Users\andre\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsforecast\core.py:26: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
C:\Users\andre\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsforecast\utils.py:236: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  "ds": pd.date_range(start="1949-01-01", periods=len(AirPassengers), freq="M"),


In [9]:
TEST_PERIODS = 2

In [1]:
Y_df_load = pd.read_parquet("test_train_data.parquet")

## Build training data

In [5]:
levels = [
    ["market"],
    ["market", "top_level_SIC_code"],
    ["market", "top_level_SIC_code", "second_level_SIC_code"],
    ["market", "top_level_SIC_code", "second_level_SIC_code", "ticker"],
    ["market", "top_level_SIC_code", "second_level_SIC_code", "ticker", 'metric']
]

In [6]:
Y_df, S_df, tags = aggregate(df=Y_df_load, spec=levels)

In [ ]:
#S_df.to_parquet('S_df.parquet')

In [ ]:
#with open('tags.pickle', 'wb') as out:
#    pickle.dump(tags, out)

In [7]:
Y_df = Y_df.reset_index()

In [10]:
Y_test_df = Y_df.groupby("unique_id").tail(TEST_PERIODS)
Y_train_df = Y_df.drop(Y_test_df.index)
Y_test_df = Y_test_df.set_index("unique_id")

## Base Forecasts

In [11]:
fcst = StatsForecast(models=[AutoARIMA(season_length=4)], freq="QE", n_jobs=-1) 

In [ ]:
Y_hat_df = fcst.forecast(
    df=Y_train_df,
    h=TEST_PERIODS,
    fitted=True,
)

In [ ]:
Y_hat_df.to_parquet('Y_hat_df.parquet')

In [ ]:
Y_fitted_df = fcst.forecast_fitted_values()

In [ ]:
Y_fitted_df.to_parquet('Y_fitted_df.parquet')

In [ ]:
fcst.plot(
    Y_train_df,
    Y_hat_df,
    models=["AutoARIMA"],
    unique_ids=["0"],
)

## Hierarchical reconciliation

In [ ]:
hrec = HierarchicalReconciliation(
    reconcilers=[
        #BottomUp(),
        # MinTrace(method="mint_shrink"),
        #MinTrace(method="ols"),
        TopDown(method='forecast_proportions')
    ]
)

In [ ]:
Y_rec_df = hrec.reconcile(Y_hat_df=Y_hat_df, Y_df=Y_fitted_df, S=S_df, tags=tags)

In [ ]:
plot_df = pd.concat(
    [
        Y_df.reset_index().set_index(["unique_id", "ds"]),
        Y_rec_df.set_index("ds", append=True),
    ],
    axis=1,
)
plot_df = plot_df.reset_index("ds")

In [ ]:
show_grid(plot_df[plot_df.index.str.startswith("6")].query('ds == "2024-06-30"'))

In [ ]:
hplot = HierarchicalPlot(S=S_df, tags=tags)
hplot.plot_series(
    series="0",
    Y_df=plot_df,
    models=[
        "y",
        "AutoARIMA",
        "AutoARIMA/TopDown_method-forecast_proportions",
    ],
    level=[80],
)

In [ ]:
hplot = HierarchicalPlot(S=S_df, tags=tags)
difference_df = Y_hat_df.merge(Y_test_df, on=["unique_id", "ds"])[
    ["ds", "AutoARIMA", "y"]
]

# hplot.plot_hierarchical_predictions_gap(Y_hat_df.rename({"Naive": "y"}, axis=1))

hplot.plot_hierarchical_predictions_gap(difference_df)

# Evaluation
## https://github.com/Nixtla/hierarchicalforecast/blob/2296c259542dbd906cfba4b8345c3b72148dad79/nbs/examples/TourismSmall.ipynb#L463

In [ ]:
evaluator = HierarchicalEvaluation(evaluators=[mse])
evaluation = evaluator.evaluate(
    Y_hat_df=Y_rec_df,
    Y_test_df=Y_test_df,
    tags=tags,
    benchmark="AutoARIMA",
)

In [ ]:
evaluation.T

In [ ]:
show_grid(
    Y_rec_df.set_index(["ds"], append=True)
    .sub(Y_test_df.set_index(["ds"], append=True)["y"], axis="index")[
        ["AutoARIMA", "AutoARIMA/BottomUp"]
    ]
    .query('ds == "2024-06-30"')
)